In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandahouse as ph
%matplotlib inline
import warnings

warnings.filterwarnings('ignore')

In [2]:

connection = {
    'host': '',
    'database':'',
    'user':'',
    'password':''
}


In [3]:
q = """
    SELECT exp_group, user_id,
    sum(action = 'like') as likes,
    sum(action = 'view') as views,
    likes/views as ctr
    FROM {db}.feed_actions
    WHERE toDate(time) between '2023-11-18' and '2023-11-24'
    and exp_group in (0,1,2,3)
    GROUP BY exp_group, user_id
"""
df = ph.read_clickhouse(q, connection=connection)
df

,exp_group,user_id,likes,views,ctr
0,1,18392,7,32,0.218750
1,3,115383,12,44,0.272727
2,2,131473,9,108,0.083333
3,3,123580,2,11,0.181818
4,2,32420,13,44,0.295455
...,...,...,...,...,...
39814,0,27030,12,64,0.187500
39815,3,122758,20,118,0.169492
39816,1,132567,52,326,0.159509
39817,3,122190,19,65,0.292308


In [5]:
CTRControl_3 = df[df.exp_group == 3].likes.sum()/df[df.exp_group == 3].views.sum()
CTRControl_2 = df[df.exp_group == 2].likes.sum()/df[df.exp_group == 2].views.sum()

In [6]:
CTRControl_3, CTRControl_2

(0.2262072573713517, 0.20025051027061783)

In [9]:
linearized_likes_0 = df[df.exp_group == 0].likes - CTRControl_3 * df[df.exp_group == 0].views
linearized_likes_3 = df[df.exp_group == 3].likes - CTRControl_3 * df[df.exp_group == 3].views
linearized_likes_1 = df[df.exp_group == 1].likes - CTRControl_2 * df[df.exp_group == 1].views
linearized_likes_2 = df[df.exp_group == 2].likes - CTRControl_2 * df[df.exp_group == 2].views

# t-test


## Группы 0 и 3

In [11]:
from scipy.stats import ttest_ind
print('t-test Линеаризированные лайки')
print(ttest_ind(linearized_likes_3, linearized_likes_0,  equal_var = False))

print('\nt-test Обычный ctr')
print(ttest_ind(df[df.exp_group == 3].ctr, df[df.exp_group == 0].ctr, equal_var = False))

t-test Линеаризированные лайки
Ttest_indResult(statistic=15.968566174822874, pvalue=4.788878529845026e-57)

t-test Обычный ctr
Ttest_indResult(statistic=13.935320516755823, pvalue=6.216047483057877e-44)




## Группы 1 и 2

In [12]:
print('t-test Линеаризированные лайки')
print(ttest_ind(linearized_likes_2, linearized_likes_1,  equal_var = False))

print('\nt-test Обычный ctr')
print(ttest_ind(df[df.exp_group == 2].ctr, df[df.exp_group == 1].ctr, equal_var = False))

t-test Линеаризированные лайки
Ttest_indResult(statistic=-5.9970015685163105, pvalue=2.057830534124001e-09)

t-test Обычный ctr
Ttest_indResult(statistic=-0.40514919131127564, pvalue=0.685373331140751)


## Выводы
Для групп 0 и 3 t-test сработал практически одинаково, но все же p-value для метрики линеаризиованных лайков показало меньшее значение, по сравнению с обычным CTR.

Что касается групп 1 и 2, различие в методах явно видно. Для обычного ctr t-test говорил о том, что нет статистически значимого отклонения. Но для метрики линеаризованных лайков t-test отвергает нулевую гипотезу о равенстве средних. 